In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import os
import win32com.client as win32
from datetime import datetime

# ★★★ openpyxl 임포트 ★★★
import openpyxl
from openpyxl.styles import Alignment, Font, Border, Side

class LibraryApp:
    def __init__(self, root):
        self.root = root
        self.root.title("김제학생교육문화관 도서관일지")

        # 모든 Entry 위젯을 담아둘 딕셔너리
        self.entries = {}

        # 날짜 선택용 콤보박스 리스트
        self.years = [str(y) for y in range(2020, 2031)]   # 2020 ~ 2030
        self.months = [str(m) for m in range(1, 13)]       # 1 ~ 12
        self.days = [str(d) for d in range(1, 32)]         # 1 ~ 31
        self.weekdays = ["월", "화", "수", "목", "금", "토", "일"]

        # 1) 큰 제목
        self.create_title_section()
        
        # 2) 화면 상단: 날짜 선택 + 첫번째 테이블
        self.create_header_and_table1()

        # 3) 두 번째 표: 관외대출도서
        self.create_table2()

        # 4) 세 번째 표: 자료실 이용자
        self.create_table3()

        # 5) 네 번째 표: 정보봉사(책바다/책나래)
        self.create_table4()

        # 6) 하단 버튼들 (저장, Excel 열기, 초기화, 출력)
        self.create_buttons()

    def create_title_section(self):
        """
        상단에 제목만 표시하는 영역입니다.
        """
        title_frame = ttk.Frame(self.root, padding=10)
        title_frame.pack(fill="x", padx=5, pady=(5, 0))
        
        title_label = ttk.Label(title_frame,
                                text="김제학생교육문화관 도서관일지",
                                font=("맑은 고딕", 18, "bold"),
                                anchor="center")
        title_label.pack(fill="x")
        
        return title_frame
    
    # ─────────────────────────────────────────────────────────────────────
    def create_header_and_table1(self):
        """
        상단 한 행에는 왼쪽에 날짜 선택 위젯, 오른쪽에 미니 테이블(결재/담당자/담당)을 배치하고,
        그 아래에 큰 표(table1)를 전체 폭에 걸쳐 배치하는 함수.
        
        미니 테이블 구성:
          - 첫 행: [결재] (왼쪽, 3행 병합), [담당자], [담당]
          - 두 번째 행: 입력 위젯을 각 열에 배치하고, 이 위젯은 rowspan=2로 병합하여 총 2개의 입력 필드만 사용.
        """
        # ─────────────────────────────
        # 전체 컨테이너
        self.container = ttk.Frame(self.root)
        self.container.pack(fill="both", expand=True, padx=5, pady=5)
        
        # ─────────────────────────────
        # 상단 헤더 행: 날짜 선택 (왼쪽) + 미니 테이블 (오른쪽)
        # ─────────────────────────────
        header_row = ttk.Frame(self.container)
        header_row.pack(fill="x", padx=5, pady=5)
        
        # [1] 왼쪽: 날짜 선택 영역
        date_frame = ttk.Frame(header_row)
        date_frame.pack(side="left", anchor="w")
        ttk.Label(date_frame, text="연:").pack(side=tk.LEFT, padx=2)
        self.year_cmb = ttk.Combobox(date_frame, values=self.years, width=5)
        self.year_cmb.pack(side=tk.LEFT, padx=2)
        current_year_str = str(datetime.now().year)
        if current_year_str in self.years:
            self.year_cmb.current(self.years.index(current_year_str))
        else:
            self.year_cmb.current(0)
        ttk.Label(date_frame, text="월:").pack(side=tk.LEFT, padx=2)
        self.month_cmb = ttk.Combobox(date_frame, values=self.months, width=3)
        self.month_cmb.pack(side=tk.LEFT, padx=2)
        self.month_cmb.current(datetime.now().month - 1)
        ttk.Label(date_frame, text="일:").pack(side=tk.LEFT, padx=2)
        self.day_cmb = ttk.Combobox(date_frame, values=self.days, width=3)
        self.day_cmb.pack(side=tk.LEFT, padx=2)
        self.day_cmb.current(datetime.now().day - 1)
        ttk.Label(date_frame, text="요일:").pack(side=tk.LEFT, padx=2)
        self.weekday_cmb = ttk.Combobox(date_frame, values=self.weekdays, width=3)
        self.weekday_cmb.pack(side=tk.LEFT, padx=2)
        today_weekday = datetime.now().weekday()  # 0=월, 6=일
        self.weekday_cmb.current(today_weekday % 7)
        
        # [2] 오른쪽: 미니 테이블 영역 (결재/담당자/담당)
        mini_table_frame = ttk.Frame(header_row, borderwidth=2, relief="solid")
        mini_table_frame.pack(side="right", anchor="ne", padx=5, pady=5)
        # --- 첫 행: 헤더 ---
        # "결재" : 0행, 0열, rowspan=3 (변경 없음)
        label_approval = ttk.Label(mini_table_frame, text="결재", borderwidth=1, relief="solid", anchor="center")
        label_approval.grid(row=0, column=0, rowspan=3, sticky="nsew", padx=1, pady=1)
        # "담당자" : 0행, 1열
        label_person = ttk.Label(mini_table_frame, text="담당자", borderwidth=1, relief="solid", anchor="center")
        label_person.grid(row=0, column=1, sticky="nsew", padx=1, pady=1)
        # "담당" : 0행, 2열
        label_resp = ttk.Label(mini_table_frame, text="담당", borderwidth=1, relief="solid", anchor="center")
        label_resp.grid(row=0, column=2, sticky="nsew", padx=1, pady=1)
        # --- 두 번째 행: 입력 필드 ---
        # 각 입력 필드는 2행을 차지하도록 rowspan=2와 ipady 옵션을 추가하여 높이를 키웁니다.
        self.person_entry = ttk.Entry(mini_table_frame, width=10)
        self.person_entry.grid(row=1, column=1, rowspan=2, sticky="nsew", padx=1, pady=1, ipady=10)
        self.responsibility_entry = ttk.Entry(mini_table_frame, width=10)
        self.responsibility_entry.grid(row=1, column=2, rowspan=2, sticky="nsew", padx=1, pady=1, ipady=10)

        
        # ─────────────────────────────
        # 아래 영역: 큰 표(table1)
        # ─────────────────────────────
        table_frame = ttk.Frame(self.container, borderwidth=2)
        table_frame.pack(fill="both", padx=5, pady=(0, 0))
        
        # 예시: table1 구성 (열람실·문화관 교육생·개관일수)
        # 왼쪽 병합: "열람실 이용자"
        lbl_열람 = ttk.Label(table_frame, text="열람실 이용자", borderwidth=1, relief="solid", anchor="center")
        lbl_열람.grid(row=0, column=0, rowspan=3, sticky="nsew", padx=1, pady=1)
        # 헤더 행: "구분", "학생", "일반", "합계"
        ttk.Label(table_frame, text="구분", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=1, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="학생", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="일반", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=3, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="합계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=4, sticky="nsew", padx=1, pady=1)
        # 중간 병합: "문화관 교육생"
        lbl_문화관 = ttk.Label(table_frame, text="문화관 교육생", borderwidth=1, relief="solid", anchor="center")
        lbl_문화관.grid(row=0, column=5, rowspan=3, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="학생교육", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=6, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="평생교육", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=7, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="시설견학", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=8, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="합계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=0, column=9, sticky="nsew", padx=1, pady=1)
        # 우측 병합: "개관일수"
        lbl_개관 = ttk.Label(table_frame, text="개관일수", borderwidth=1, relief="solid", anchor="center")
        lbl_개관.grid(row=0, column=10, columnspan=2, sticky="nsew", padx=1, pady=1)
        # 금일/누계 라벨
        ttk.Label(table_frame, text="금일", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=1, sticky="nsew", padx=1, pady=1)
        ttk.Label(table_frame, text="누계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=2, column=1, sticky="nsew", padx=1, pady=1)
        
        # 금일(열람실) 입력 필드 (예시)
        keys_gumil_열람 = ["금일_열람_학생", "금일_열람_일반", "금일_열람_합계"]
        for i, key in enumerate(keys_gumil_열람, start=2):
            ent = ttk.Entry(table_frame, width=8)
            ent.grid(row=1, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 누계(열람실) 입력 필드 (예시)
        keys_nu_열람 = ["누계_열람_학생", "누계_열람_일반", "누계_열람_합계"]
        for i, key in enumerate(keys_nu_열람, start=2):
            ent = ttk.Entry(table_frame, width=8)
            ent.grid(row=2, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 금일(문화관 교육생) 입력 필드 (예시)
        keys_gumil_문화 = ["금일_문화_학생교육", "금일_문화_평생교육", "금일_문화_시설견학", "금일_문화_합계"]
        for i, key in enumerate(keys_gumil_문화, start=6):
            ent = ttk.Entry(table_frame, width=8)
            ent.grid(row=1, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 누계(문화관 교육생) 입력 필드 (예시)
        keys_nu_문화 = ["누계_문화_학생교육", "누계_문화_평생교육", "누계_문화_시설견학", "누계_문화_합계"]
        for i, key in enumerate(keys_nu_문화, start=6):
            ent = ttk.Entry(table_frame, width=8)
            ent.grid(row=2, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 열람실/자료실 개관일수 입력 필드 (예시)
        ttk.Label(table_frame, text="열람실", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=10, sticky="nsew", padx=1, pady=1)
        ent_open1 = ttk.Entry(table_frame, width=8)
        ent_open1.grid(row=1, column=11, sticky="nsew", padx=1, pady=1)
        self.entries["금일_개관_열람실"] = ent_open1
        ttk.Label(table_frame, text="자료실", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=2, column=10, sticky="nsew", padx=1, pady=1)
        ent_open2 = ttk.Entry(table_frame, width=8)
        ent_open2.grid(row=2, column=11, sticky="nsew", padx=1, pady=1)
        self.entries["누계_개관_자료실"] = ent_open2
        
        for c in range(12):
            table_frame.columnconfigure(c, weight=1)


    # ─────────────────────────────────────────────────────────────────────
    # (3) 두 번째 표: 관외대출도서
    # ─────────────────────────────────────────────────────────────────────
    def create_table2(self):
        # LabelFrame 대신 일반 Frame 사용 (text 인자 없음)
        loan_frame = ttk.Frame(self.container)
        loan_frame.pack(padx=10, pady=(0, 5), fill=tk.X)
        
        categories = ["총류", "철학", "종교", "사회과학", "자연과학",
                      "기술과학", "예술", "언어", "문학", "역사", "계"]
        
        # 가장 왼쪽 0열에 "관외 대출 도서" Label 추가 (행 전체 span: 7행(0~6))
        lbl_title = ttk.Label(loan_frame, text="관외 대출 도서", borderwidth=1, relief="solid", anchor="center")
        lbl_title.grid(row=0, column=0, rowspan=7, sticky="nsew", padx=1, pady=1)
        
        # (0) 헤더행: 원래 구분은 0열에 있었으나, 이제 오른쪽으로 한 칸 이동 (열 1~2)
        lbl_gubun = ttk.Label(loan_frame, text="구분", borderwidth=1, relief="solid", anchor="center")
        lbl_gubun.grid(row=0, column=1, columnspan=2, sticky="nsew", padx=1, pady=1)
        
        # 각 카테고리 라벨: 원래 start=2 → 이제 start=3
        for i, cat in enumerate(categories, start=3):
            ttk.Label(loan_frame, text=cat, borderwidth=1, relief="solid", anchor="center")\
                .grid(row=0, column=i, sticky="nsew", padx=1, pady=1)
        
        # (1) 일반자료실: 금일/누계 (행 1~2)
        # "일반자료실"은 원래 row=1, column=0, rowspan=2 → 이제 row=1, column=1
        ttk.Label(loan_frame, text="일반자료실", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=1, rowspan=2, sticky="nsew", padx=1, pady=1)
        
        ttk.Label(loan_frame, text="금일", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=1, column=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(loan_frame, text="누계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=2, column=2, sticky="nsew", padx=1, pady=1)
        
        # 일반자료실 - 금일
        for i, cat in enumerate(categories, start=3):
            key = f"관외_일반_금일_{cat}"
            ent = ttk.Entry(loan_frame, width=6)
            ent.grid(row=1, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 일반자료실 - 누계
        for i, cat in enumerate(categories, start=3):
            key = f"관외_일반_누계_{cat}"
            ent = ttk.Entry(loan_frame, width=6)
            ent.grid(row=2, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # (2) 어린이자료실: 금일/누계 (행 3~4)
        # "어린이자료실\n(책펴락)" 원래 row=3, column=0, rowspan=2 → 이제 row=3, column=1
        ttk.Label(loan_frame, text="어린이자료실\n(책펴락)", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=3, column=1, rowspan=2, sticky="nsew", padx=1, pady=1)
        
        ttk.Label(loan_frame, text="금일", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=3, column=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(loan_frame, text="누계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=4, column=2, sticky="nsew", padx=1, pady=1)
        
        # 어린이자료실 - 금일
        for i, cat in enumerate(categories, start=3):
            key = f"관외_어린이_금일_{cat}"
            ent = ttk.Entry(loan_frame, width=6)
            ent.grid(row=3, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 어린이자료실 - 누계
        for i, cat in enumerate(categories, start=3):
            key = f"관외_어린이_누계_{cat}"
            ent = ttk.Entry(loan_frame, width=6)
            ent.grid(row=4, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # (3) 합계: 금일/누계 (행 5~6)
        # "합계" 원래 row=5, column=0, rowspan=2 → 이제 row=5, column=1
        ttk.Label(loan_frame, text="합계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=5, column=1, rowspan=2, sticky="nsew", padx=1, pady=1)
        
        ttk.Label(loan_frame, text="금일", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=5, column=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(loan_frame, text="누계", borderwidth=1, relief="solid", anchor="center")\
            .grid(row=6, column=2, sticky="nsew", padx=1, pady=1)
        
        # 합계 - 금일
        for i, cat in enumerate(categories, start=3):
            key = f"관외_합계_금일_{cat}"
            ent = ttk.Entry(loan_frame, width=6)
            ent.grid(row=5, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 합계 - 누계
        for i, cat in enumerate(categories, start=3):
            key = f"관외_합계_누계_{cat}"
            ent = ttk.Entry(loan_frame, width=6)
            ent.grid(row=6, column=i, sticky="nsew", padx=1, pady=1)
            self.entries[key] = ent
        
        # 전체 열의 수는: 1 (제목) + 2 (구분 부분) + len(categories)
        total_columns = 1 + 2 + len(categories)
        for c in range(total_columns):
            loan_frame.columnconfigure(c, weight=1)


    # ─────────────────────────────────────────────────────────────────────
    # (4) 세 번째 표: 자료실 이용자
    # ─────────────────────────────────────────────────────────────────────
    # ─────────────────────────────────────────────────────────────────────
# (4) 세 번째 표: 자료실 이용자
# ─────────────────────────────────────────────────────────────────────
    def create_table3(self):
        # LabelFrame 대신 일반 Frame 사용
        new_frame = ttk.Frame(self.container)
        new_frame.pack(fill=tk.X, padx=10, pady=5)
        
        # 가장 왼쪽 0열에 "자료실 이용자" Label 추가 (전체 행 span)
        # 전체 행 수는 6행(0~5)
        lbl_title = ttk.Label(new_frame, text="자료실 이용자", borderwidth=1, relief="solid", anchor="center")
        lbl_title.grid(row=0, column=0, rowspan=6, sticky="nsew", padx=1, pady=1)
        
        # 열 인덱스를 한 칸씩 오른쪽으로 이동 (모든 열에 +1)
        # 첫 번째 행
        lbl_gubun = ttk.Label(new_frame, text="구분", borderwidth=1, relief="solid", anchor="center")
        lbl_gubun.grid(row=0, column=1, rowspan=2, sticky="nsew", padx=1, pady=1)
        
        lbl_normal = ttk.Label(new_frame, text="일반자료실", borderwidth=1, relief="solid", anchor="center")
        lbl_normal.grid(row=0, column=2, rowspan=2, sticky="nsew", padx=1, pady=1)
        
        lbl_child = ttk.Label(new_frame, text="어린이자료실(책펴락)", borderwidth=1, relief="solid", anchor="center")
        lbl_child.grid(row=0, column=3, rowspan=2, sticky="nsew", padx=1, pady=1)
        
        # 디지털 코너
        lbl_digital = ttk.Label(new_frame, text="디지털 코너", borderwidth=1, relief="solid", anchor="center")
        lbl_digital.grid(row=0, column=4, columnspan=3, sticky="nsew", padx=1, pady=1)
        
        lbl_sum_col = ttk.Label(new_frame, text="합계", borderwidth=1, relief="solid", anchor="center")
        lbl_sum_col.grid(row=0, column=7, rowspan=2, sticky="nsew", padx=1, pady=1)
        
        # 도서관회원가입
        lbl_membership = ttk.Label(new_frame, text="도서관회원가입", borderwidth=1, relief="solid", anchor="center")
        lbl_membership.grid(row=0, column=8, columnspan=4, sticky="nsew", padx=1, pady=1)
        
        # 두 번째 행 (하위)
        lbl_pc = ttk.Label(new_frame, text="PC", borderwidth=1, relief="solid", anchor="center")
        lbl_pc.grid(row=1, column=4, sticky="nsew", padx=1, pady=1)
        
        lbl_vod = ttk.Label(new_frame, text="VOD", borderwidth=1, relief="solid", anchor="center")
        lbl_vod.grid(row=1, column=5, sticky="nsew", padx=1, pady=1)
        
        lbl_sogae = ttk.Label(new_frame, text="소계", borderwidth=1, relief="solid", anchor="center")
        lbl_sogae.grid(row=1, column=6, sticky="nsew", padx=1, pady=1)
        
        lbl_stu = ttk.Label(new_frame, text="학생", borderwidth=1, relief="solid", anchor="center")
        lbl_stu.grid(row=1, column=8, sticky="nsew", padx=1, pady=1)
        
        lbl_gen = ttk.Label(new_frame, text="일반", borderwidth=1, relief="solid", anchor="center")
        lbl_gen.grid(row=1, column=9, sticky="nsew", padx=1, pady=1)
        
        lbl_sum2 = ttk.Label(new_frame, text="합계", borderwidth=1, relief="solid", anchor="center")
        lbl_sum2.grid(row=1, column=10, sticky="nsew", padx=1, pady=1)
        
        lbl_total = ttk.Label(new_frame, text="총회원수", borderwidth=1, relief="solid", anchor="center")
        lbl_total.grid(row=1, column=11, sticky="nsew", padx=1, pady=1)
        
        # row=2~3: 열람자(금일/누계)
        row_labels_viewer = [
            ("열람자(금일)", "열람자_금일"),
            ("열람자(누계)", "열람자_누계"),
        ]
        for i, (lbl_text, row_key) in enumerate(row_labels_viewer):
            r = 2 + i
            lbl_left = ttk.Label(new_frame, text=lbl_text, borderwidth=1, relief="solid", anchor="center")
            lbl_left.grid(row=r, column=1, sticky="nsew", padx=1, pady=1)
            for c in range(2, 12):  # 열 인덱스를 1 증가
                ent = ttk.Entry(new_frame, width=8)
                ent.grid(row=r, column=c, sticky="nsew", padx=1, pady=1)
                dict_key = f"자료실_{row_key}_col{c-1}"  # 키는 기존 방식 유지를 위해 c-1
                self.entries[dict_key] = ent
        
        # row=4~5: 대출자(금일/누계)
        row_labels_borrower = [
            ("대출자(금일)", "대출자_금일"),
            ("대출자(누계)", "대출자_누계"),
        ]
        for i, (lbl_text, row_key) in enumerate(row_labels_borrower):
            r = 4 + i
            lbl_left = ttk.Label(new_frame, text=lbl_text, borderwidth=1, relief="solid", anchor="center")
            lbl_left.grid(row=r, column=1, sticky="nsew", padx=1, pady=1)
            for c in range(2, 12):  # 열 인덱스를 1 증가
                ent = ttk.Entry(new_frame, width=8)
                ent.grid(row=r, column=c, sticky="nsew", padx=1, pady=1)
                dict_key = f"자료실_{row_key}_col{c-1}"  # 키는 기존 방식 유지를 위해 c-1
                self.entries[dict_key] = ent
            
            # 도서관회원가입 영역
            if r == 4:
                lbl_day_user = ttk.Label(new_frame, text="금일 이용자수", borderwidth=1, relief="solid", anchor="center")
                lbl_day_user.grid(row=4, column=8, columnspan=2, sticky="nsew", padx=1, pady=1)
                ent_day_user = ttk.Entry(new_frame, width=16)
                ent_day_user.grid(row=4, column=10, columnspan=2, sticky="nsew", padx=1, pady=1)
                self.entries["도서관회원가입_금일이용자수"] = ent_day_user
            else:
                lbl_total_user = ttk.Label(new_frame, text="총 이용자수", borderwidth=1, relief="solid", anchor="center")
                lbl_total_user.grid(row=5, column=8, columnspan=2, sticky="nsew", padx=1, pady=1)
                ent_total_user = ttk.Entry(new_frame, width=16)
                ent_total_user.grid(row=5, column=10, columnspan=2, sticky="nsew", padx=1, pady=1)
                self.entries["도서관회원가입_총이용자수"] = ent_total_user
        
        for c in range(12):  # 열 개수 증가 (11 -> 12)
            new_frame.columnconfigure(c, weight=1)

    # ─────────────────────────────────────────────────────────────────────
    # (5) 네 번째 표: 정보봉사 (책바다/책나래)
    # ─────────────────────────────────────────────────────────────────────
    def create_table4(self):
        service_frame = ttk.LabelFrame(self.container, text="정보봉사")
        service_frame.pack(fill=tk.X, padx=10, pady=5)
        
        # 헤더
        lbl_booksea = ttk.Label(service_frame, text="책바다", borderwidth=1, relief="solid", anchor="center")
        lbl_booksea.grid(row=0, column=0, columnspan=2, sticky="nsew", padx=1, pady=1)

        lbl_booknara = ttk.Label(service_frame, text="책나래", borderwidth=1, relief="solid", anchor="center")
        lbl_booknara.grid(row=0, column=2, columnspan=2, sticky="nsew", padx=1, pady=1)

        lbl_info_day = ttk.Label(service_frame, text="금일정보봉사\n(자동합계)", borderwidth=1, relief="solid", anchor="center")
        lbl_info_day.grid(row=0, column=4, sticky="nsew", padx=1, pady=1)

        lbl_info_total = ttk.Label(service_frame, text="합계(누계)\n(자동합계)", borderwidth=1, relief="solid", anchor="center")
        lbl_info_total.grid(row=0, column=5, sticky="nsew", padx=1, pady=1)

        # 하위 헤더
        ttk.Label(service_frame, text="금일", borderwidth=1, relief="solid", anchor="center").grid(row=1, column=0, sticky="nsew", padx=1, pady=1)
        ttk.Label(service_frame, text="누계", borderwidth=1, relief="solid", anchor="center").grid(row=1, column=1, sticky="nsew", padx=1, pady=1)

        ttk.Label(service_frame, text="금일", borderwidth=1, relief="solid", anchor="center").grid(row=1, column=2, sticky="nsew", padx=1, pady=1)
        ttk.Label(service_frame, text="누계", borderwidth=1, relief="solid", anchor="center").grid(row=1, column=3, sticky="nsew", padx=1, pady=1)

        ttk.Label(service_frame, text="금일", borderwidth=1, relief="solid", anchor="center").grid(row=1, column=4, sticky="nsew", padx=1, pady=1)
        ttk.Label(service_frame, text="누계", borderwidth=1, relief="solid", anchor="center").grid(row=1, column=5, sticky="nsew", padx=1, pady=1)

        # 입력칸
        self.booksea_day_entry = ttk.Entry(service_frame, width=8)
        self.booksea_day_entry.grid(row=2, column=0, sticky="nsew", padx=1, pady=1)
        self.entries["책바다_금일"] = self.booksea_day_entry
        
        self.booksea_total_entry = ttk.Entry(service_frame, width=8)
        self.booksea_total_entry.grid(row=2, column=1, sticky="nsew", padx=1, pady=1)
        self.entries["책바다_누계"] = self.booksea_total_entry
        
        self.booknara_day_entry = ttk.Entry(service_frame, width=8)
        self.booknara_day_entry.grid(row=2, column=2, sticky="nsew", padx=1, pady=1)
        self.entries["책나래_금일"] = self.booknara_day_entry
        
        self.booknara_total_entry = ttk.Entry(service_frame, width=8)
        self.booknara_total_entry.grid(row=2, column=3, sticky="nsew", padx=1, pady=1)
        self.entries["책나래_누계"] = self.booknara_total_entry
        
        self.info_day_entry = ttk.Entry(service_frame, width=8, state="readonly")
        self.info_day_entry.grid(row=2, column=4, sticky="nsew", padx=1, pady=1)
        
        self.info_total_entry = ttk.Entry(service_frame, width=8, state="readonly")
        self.info_total_entry.grid(row=2, column=5, sticky="nsew", padx=1, pady=1)
        
        # 이벤트 바인딩(자동 합계)
        self.booksea_day_entry.bind("<KeyRelease>", self.update_service_sum)
        self.booksea_total_entry.bind("<KeyRelease>", self.update_service_sum)
        self.booknara_day_entry.bind("<KeyRelease>", self.update_service_sum)
        self.booknara_total_entry.bind("<KeyRelease>", self.update_service_sum)

        for c in range(6):
            service_frame.columnconfigure(c, weight=1)

    def update_service_sum(self, event=None):
        try:
            booksea_day = float(self.booksea_day_entry.get())
        except ValueError:
            booksea_day = 0.0

        try:
            booknara_day = float(self.booknara_day_entry.get())
        except ValueError:
            booknara_day = 0.0

        day_sum = booksea_day + booknara_day

        try:
            booksea_total = float(self.booksea_total_entry.get())
        except ValueError:
            booksea_total = 0.0

        try:
            booknara_total = float(self.booknara_total_entry.get())
        except ValueError:
            booknara_total = 0.0

        total_sum = booksea_total + booknara_total

        self.info_day_entry.config(state="normal")
        self.info_day_entry.delete(0, tk.END)
        self.info_day_entry.insert(0, str(day_sum))
        self.info_day_entry.config(state="readonly")

        self.info_total_entry.config(state="normal")
        self.info_total_entry.delete(0, tk.END)
        self.info_total_entry.insert(0, str(total_sum))
        self.info_total_entry.config(state="readonly")

    # ─────────────────────────────────────────────────────────────────────
    # (6) 하단 버튼
    # ─────────────────────────────────────────────────────────────────────
    def create_buttons(self):
        btn_frame = ttk.Frame(self.container, padding=10)
        btn_frame.pack()
    
        ttk.Button(btn_frame, text="저장", command=self.save_to_excel).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="Excel 열기", command=self.open_excel).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="초기화", command=self.clear_entries).pack(side=tk.LEFT, padx=5)
        ttk.Button(btn_frame, text="출력", command=self.print_excel).pack(side=tk.LEFT, padx=5)

    # ─────────────────────────────────────────────────────────────────────
    # ★★★ 핵심: openpyxl 로 수정한 save_to_excel() ★★★
    # ─────────────────────────────────────────────────────────────────────
    def save_to_excel(self):
        """
        openpyxl을 사용하여 날짜별 시트에 데이터를 저장하는 예시.
        - 파일이 없으면 새로 생성
        - 'YYYY-MM-DD' 이름의 시트가 없으면 새 시트 생성
        - 해당 시트에 GUI 데이터(Entry.get()) 기록
        """
        excel_file = "data_input.xlsx"
        
        # 1) 날짜 문자열(시트 이름) 만들기
        #   예) "2025-03-29 (토)" 이런 식으로도 가능.
        date_str = f"{self.year_cmb.get()}-{self.month_cmb.get().zfill(2)}-{self.day_cmb.get().zfill(2)} ({self.weekday_cmb.get()}요일)"

        # 2) Workbook 불러오기 (없으면 새로 생성)
        if os.path.exists(excel_file):
            wb = openpyxl.load_workbook(excel_file)
        else:
            wb = openpyxl.Workbook()

        # 3) 같은 날짜의 시트가 이미 있으면 그 시트를 사용, 없으면 새 시트 생성
        if date_str in wb.sheetnames:
            ws = wb[date_str]
            # 기존 내용을 지우고 다시 쓰고 싶으면, 아래처럼 한 번 전체 삭제할 수도 있음.
            # for row in ws.iter_rows():
            #     for cell in row:
            #         cell.value = None
        else:
            ws = wb.create_sheet(title=date_str)

        # 4) 셀에 기록하기 전에, 스타일(폰트, 정렬, 테두리) 세팅 준비
        #    openpyxl은 xlsxwriter처럼 '포맷' 객체를 미리 만들어 쓰지 않고,
        #    cell마다 스타일을 지정하거나, NamedStyle 등을 활용.
        align_center = Alignment(horizontal="center", vertical="center", wrap_text=True)
        thin_border = Border(left=Side(style='thin'), 
                             right=Side(style='thin'), 
                             top=Side(style='thin'), 
                             bottom=Side(style='thin'))

        # 5) 일단 A1 등에 날짜/제목 쓰기 (간단 버전)
        ws["A1"].value = "날짜"
        ws["A1"].alignment = align_center
        ws["B1"].value = date_str
        ws["B1"].alignment = align_center

        ws.merge_cells("A2:K2")
        ws["A2"].value = "김제학생교육문화관 도서관일지"
        ws["A2"].alignment = Alignment(horizontal="center", vertical="center")
        ws["A2"].font = Font(size=16, bold=True)

        # ───────────────────────────────────────────────────────────
        # 예시로, 표1(열람실/문화관...) 일부만 간단히 적는 방식
        # 실제로는 xlsxwriter에서 하던 것처럼 테이블 구조에 맞춰
        # merge_cells와 각 셀 위치에 데이터를 기록해야 함
        # ───────────────────────────────────────────────────────────
        
        row_idx = 4  # 표를 시작할 행
        
        # (예) "열람실 이용자"
        ws.cell(row=row_idx, column=1, value="금일_열람_학생").alignment = align_center
        ws.cell(row=row_idx, column=2, value=self.entries["금일_열람_학생"].get()).alignment = align_center
        
        ws.cell(row=row_idx, column=3, value="금일_열람_일반").alignment = align_center
        ws.cell(row=row_idx, column=4, value=self.entries["금일_열람_일반"].get()).alignment = align_center
        
        ws.cell(row=row_idx, column=5, value="금일_열람_합계").alignment = align_center
        ws.cell(row=row_idx, column=6, value=self.entries["금일_열람_합계"].get()).alignment = align_center

        # ... 이런 식으로 나머지 필드도 적절한 셀에 써넣는다.
        # 여기서는 예시로 몇 칸만 시연.
        
        # (실제로는 xlsxwriter 때처럼 머지 셀과 표 레이아웃을 똑같이 구성하려면
        #  row와 column 인덱스를 적절히 이동, merge_cells(...)도 적절히 수행해 주어야 합니다.)
        
        # 셀에 일괄적으로 테두리, 정렬을 주고 싶다면 한 번 더 루프를 돌면서 적용
        max_row = ws.max_row
        max_col = ws.max_column
        for r in range(1, max_row + 1):
            for c in range(1, max_col + 1):
                cell = ws.cell(r, c)
                if cell.value is not None:  # 값이 있는 셀만 스타일 적용 (원하시면 전부 적용 가능)
                    cell.alignment = align_center
                    cell.border = thin_border

        # 6) 최종 저장
        wb.save(excel_file)
        messagebox.showinfo("저장 완료", f"'{date_str}' 시트에 데이터가 저장되었습니다.")

    # ─────────────────────────────────────────────────────────────────────
    def open_excel(self):
        """저장된 Excel 파일 열기 (Windows에서 기본 프로그램으로 열기)"""
        excel_file = "data_input.xlsx"
        if not os.path.exists(excel_file):
            messagebox.showinfo("알림", "아직 Excel 파일이 생성되지 않았습니다.")
            return
        try:
            os.startfile(os.path.abspath(excel_file))
        except Exception as e:
            messagebox.showerror("에러", f"Excel 열기 중 오류: {str(e)}")

    def clear_entries(self):
        """모든 입력 필드 초기화"""
        for entry in self.entries.values():
            entry.delete(0, tk.END)

        # 날짜 콤보박스도 현재 날짜로 재설정
        now = datetime.now()
        current_year_str = str(now.year)
        if current_year_str in self.years:
            self.year_cmb.current(self.years.index(current_year_str))
        else:
            self.year_cmb.current(0)
        self.month_cmb.current(now.month - 1)
        self.day_cmb.current(now.day - 1)
        self.weekday_cmb.current(now.weekday() % 7)

    def print_excel(self):
        """저장된 Excel 파일을 열어 바로 프린터로 출력"""
        excel_file = "data_input.xlsx"
        
        # 파일이 존재하는지 확인
        if not os.path.exists(excel_file):
            messagebox.showinfo("알림", "아직 Excel 파일이 생성되지 않았습니다.")
            return
    
        try:
            # Excel Application 객체 생성
            excel_app = win32.Dispatch("Excel.Application")
            excel_app.Visible = False  # 화면에 띄우지 않고 작업하려면 False
            # Excel 파일 열기
            wb = excel_app.Workbooks.Open(os.path.abspath(excel_file))
            
            # 일단 기본적으로 첫 번째 시트를 대상으로 인쇄하거나,
            # 혹은 특정 시트 이름("도서관일지")을 찾을 수도 있음
            # 여기서는 첫 시트로 가정
            ws = wb.Worksheets(1)
            
            # 실제 프린터로 출력
            ws.PrintOut()  
            
            # 작업 종료
            wb.Close(SaveChanges=False)
            excel_app.Quit()
            
            messagebox.showinfo("출력 완료", "Excel 파일을 프린터로 인쇄했습니다.")
            
        except Exception as e:
            messagebox.showerror("에러", f"인쇄 과정에서 오류가 발생했습니다.\n{str(e)}")

# 메인 구동부
if __name__ == "__main__":
    root = tk.Tk()
    app = LibraryApp(root)
    root.mainloop()
